In [10]:
import pyarrow as pa
import pyarrow.dataset as ds
import duckdb
from pyarrow import fs
import streamlit as st

def compaction(day):   
    duckdb.sql(f'''
        install httpfs;
        LOAD httpfs;
        set enable_progress_bar=false;
        PRAGMA enable_object_cache;
        SET enable_http_metadata_cache=true ;
        set s3_region = 'auto';
        set s3_access_key_id = "{st.secrets["aws_access_key_id_secret"]}" ;
        set s3_secret_access_key = '{st.secrets["aws_secret_access_key_secret"] }';
        set s3_endpoint = '{st.secrets["endpoint_url_secret"].replace("https://", "")}' 
        
        ''')
    file=duckdb.sql(f"""select count(distinct filename) as cnt 
                     from parquet_scan('s3://aemo/aemo/scada/data/Date={day}/*.parquet' , HIVE_PARTITIONING = 1,filename=1) """).df()
    if file[['cnt']].values[0][0] > 1 :
    
         tb=duckdb.sql(f'''select SETTLEMENTDATE,DUID,SCADAVALUE,file, cast(Date as date) as Date
                       from parquet_scan('s3://aemo/aemo/scada/data/Date={day}/*.parquet' , HIVE_PARTITIONING = 1)''').arrow()
    
         s3 = fs.S3FileSystem(region="us-east-1",
                         access_key = st.secrets["aws_access_key_id_secret"],
                         secret_key=st.secrets["aws_secret_access_key_secret"] ,
                         endpoint_override=st.secrets["endpoint_url_secret"] )
        
    
         my_schema = pa.schema([
                      pa.field('SETTLEMENTDATE', pa.timestamp('us')),
                      pa.field('DUID', pa.string()),
                      pa.field('SCADAVALUE', pa.float64()),
                      pa.field('file', pa.string()),
                      pa.field('Date', pa.date32())
                         ])
         xx=tb.cast(target_schema=my_schema)
         ds.write_dataset(xx,"aemo/aemo/scada/data/", filesystem=s3,format="parquet" , partitioning=['Date'],partitioning_flavor="hive",
         min_rows_per_group=120000,existing_data_behavior="delete_matching")
    return 'done'


In [ ]:
from datetime import datetime , timedelta
import pytz
import time
now = datetime.now(pytz.timezone('Australia/Brisbane'))
date_list =[datetime.strftime(now - timedelta(days=x), '%Y-%m-%d') for x in range(1, 30) ]
for x in date_list :
    start = time.time()
    compaction(x)
    stop = time.time()
    duration = stop-start
    print(x)
    print(duration)